In [73]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [2]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [3]:
link = pd.read_csv('./ml-latest-small/links.csv')

In [4]:
rating = pd.read_csv('./ml-latest-small/ratings.csv')

In [5]:
tags = pd.read_csv('./ml-latest-small/tags.csv')

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [8]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [9]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [14]:
movie_rating = movies.join(rating, on='movieId', lsuffix='movie_', rsuffix='rating_', how='outer')

In [15]:
movie_rating

,movieId,movieIdmovie_,title,genres,userId,movieIdrating_,rating,timestamp
0.0,1,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,3.0,4.0,9.649812e+08
1.0,2,2.0,Jumanji (1995),Adventure|Children|Fantasy,1.0,6.0,4.0,9.649822e+08
2.0,3,3.0,Grumpier Old Men (1995),Comedy|Romance,1.0,47.0,5.0,9.649838e+08
3.0,4,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,1.0,50.0,5.0,9.649829e+08
4.0,5,5.0,Father of the Bride Part II (1995),Comedy,1.0,70.0,3.0,9.649824e+08
...,...,...,...,...,...,...,...,...
NaN,100831,NaN,NaN,NaN,610.0,166534.0,4.0,1.493848e+09
NaN,100832,NaN,NaN,NaN,610.0,168248.0,5.0,1.493850e+09
NaN,100833,NaN,NaN,NaN,610.0,168250.0,5.0,1.494273e+09
NaN,100834,NaN,NaN,NaN,610.0,168252.0,5.0,1.493846e+09


In [45]:
movie_rating.dropna(inplace=True)

In [46]:
movie_rating['userId'].nunique()

397

In [47]:
stats = movie_rating[['rating', 'timestamp']].describe()
stats

,rating,timestamp
count,8113.000000,8.113000e+03
mean,3.461666,1.167825e+09
std,1.094076,2.273938e+08
min,0.500000,8.281246e+08
25%,3.000000,9.657060e+08
50%,3.500000,1.117507e+09
75%,4.000000,1.419025e+09
max,5.000000,1.537235e+09


In [48]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

1996-03-29 12:36:55
2018-09-17 20:49:14


## Split

In [34]:
movie_rating.columns

Index(['movieId', 'movieIdmovie_', 'title', 'genres', 'userId',
       'movieIdrating_', 'rating', 'timestamp'],
      dtype='object')

In [50]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(movie_rating[['userId', 'movieId', 'rating']], reader)

In [51]:
data

In [52]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [53]:
train

In [54]:
model = KNNBasic().fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
model2 = SVD().fit(train)

In [56]:
model3 = NMF().fit(train)

In [74]:
model4 = knns.KNNWithMeans().fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


## Tuning

In [76]:
model = KNNBasic()

In [78]:
train.

In [77]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(model,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train)

AttributeError: 'Trainset' object has no attribute 'raw_ratings'

In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [66]:
CV1 = surprise.model_selection.split.KFold()

In [ ]:
surprise.model_selection.split.RepeatedKFold()